In [28]:
print("Hello")

Hello


In [29]:
!lscpu

Architecture:                x86_64
  CPU op-mode(s):            32-bit, 64-bit
  Address sizes:             46 bits physical, 48 bits virtual
  Byte Order:                Little Endian
CPU(s):                      2
  On-line CPU(s) list:       0,1
Vendor ID:                   GenuineIntel
  Model name:                Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:              6
    Model:                   79
    Thread(s) per core:      2
    Core(s) per socket:      1
    Socket(s):               1
    Stepping:                0
    BogoMIPS:                4399.99
    Flags:                   fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pg
                             e mca cmov pat pse36 clflush mmx fxsr sse sse2 ss h
                             t syscall nx pdpe1gb rdtscp lm constant_tsc rep_goo
                             d nopl xtopology nonstop_tsc cpuid tsc_known_freq p
                             ni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2ap
                   

In [30]:
%%writefile hello.cpp
#include <iostream>

using namespace std;
int main() {
  cout << "Hello Colab!" << endl;
  return 0;
}

Overwriting hello.cpp


In [31]:
!ls

hello	   matmul      matmul_avx.cpp  matmul_opt      sample_data
hello.cpp  matmul_avx  matmul.cpp      matmul_opt.cpp


In [32]:
!g++ -O2 -std=c++11 hello.cpp -o hello

In [33]:
!./hello

Hello Colab!


In [34]:
%%writefile matmul.cpp
#include <iostream>
#include <chrono>
#include <cstdlib>
#include <ctime>
using namespace std;

void matmul(float* A, float* B, float* C, int n) {
  for (int i = 0; i < n; i++) {
    for (int j = 0; j < n; j++) {
      float accum = 0;
      for (int k = 0; k < n; k++) {
        accum += A[i * n + k] * B[k * n + j];
      }
      C[i * n + j] = accum;
    }
  }
}

void init(float* A, int n, int m, bool zero) {
  for (int i = 0; i < n * m; i++) {
    A[i] = zero ? 0 : rand() / float(RAND_MAX);
  }
}

int main() {
  int n = 1024;
  float* A = new float[n * n];
  float* B = new float[n * n];
  float* C = new float[n * n];

  init(A, n, n, false);
  init(B, n, n, false);
  init(C, n, n, true);

  auto start = chrono::high_resolution_clock::now();

  matmul(A, B, C, n);

  auto end = chrono::high_resolution_clock::now();
  chrono::duration<double> diff = end - start;
  cout << "Execution time: " << diff.count() << " s" << endl;
  cout << "C[0][0]: " << C[0] << endl;

  delete[] A;
  delete[] B;
  delete[] C;
}

Overwriting matmul.cpp


In [35]:
!g++ -O2 -std=c++11 matmul.cpp -o matmul

In [36]:
!./matmul

Execution time: 4.70074 s
C[0][0]: 264.159


In [37]:
%%writefile matmul_opt.cpp
#include <iostream>
#include <chrono>
#include <cstdlib>
#include <ctime>
using namespace std;


void init(float* A, int n, int m, bool zero) {
  for (int i = 0; i < n * m; i++) {
    A[i] = zero ? 0 : rand() / float(RAND_MAX);
  }
}

void matmul(float* A, float* B, float* C, int n) {
  float* Bt = new float[n * n];
  init(Bt, n, n, true);

 for (int k = 0; k < n; k++) {
  for(int j = 0; j < n; j++) {
    Bt[j * n + k] = B[k * n + j];
  }
 }
 for (int i = 0; i < n; i++) {
  for (int j = 0; j < n; j++) {
    float accum = 0;
    for (int k = 0; k < n; k++) {
      accum += A[i * n + k] * Bt[j * n + k];
    }
    C[i * n + j] = accum;
  }
 }
}

int main() {
  int n = 1024;
  float* A = new float[n * n];
  float* B = new float[n * n];
  float* C = new float[n * n];

  init(A, n, n, false);
  init(B, n, n, false);
  init(C, n, n, true);

  auto start = chrono::high_resolution_clock::now();

  matmul(A, B, C, n);

  auto end = chrono::high_resolution_clock::now();
  chrono::duration<double> diff = end - start;
  cout << "Execution time: " << diff.count() << " s" << endl;
  cout << "C[0][0]: " << C[0] << endl;

  delete[] A;
  delete[] B;
  delete[] C;
}

Overwriting matmul_opt.cpp


In [38]:
!g++ -O2 -std=c++11 matmul_opt.cpp -o matmul_opt

In [39]:
!./matmul_opt

Execution time: 1.65773 s
C[0][0]: 264.159


In [40]:
!ls-cpu

/bin/bash: line 1: ls-cpu: command not found


In [41]:
!lscpu

Architecture:                x86_64
  CPU op-mode(s):            32-bit, 64-bit
  Address sizes:             46 bits physical, 48 bits virtual
  Byte Order:                Little Endian
CPU(s):                      2
  On-line CPU(s) list:       0,1
Vendor ID:                   GenuineIntel
  Model name:                Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:              6
    Model:                   79
    Thread(s) per core:      2
    Core(s) per socket:      1
    Socket(s):               1
    Stepping:                0
    BogoMIPS:                4399.99
    Flags:                   fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pg
                             e mca cmov pat pse36 clflush mmx fxsr sse sse2 ss h
                             t syscall nx pdpe1gb rdtscp lm constant_tsc rep_goo
                             d nopl xtopology nonstop_tsc cpuid tsc_known_freq p
                             ni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2ap
                   

In [42]:
%%writefile matmul_avx.cpp
#include <iostream>
#include <chrono>
#include <cstdlib>
#include <ctime>
#include <immintrin.h>

using namespace std;


void init(float* A, int n, int m, bool zero) {
  for (int i = 0; i < n * m; i++) {
    A[i] = zero ? 0 : rand() / float(RAND_MAX);
  }
}

float hsum_avx(__m256 X) {
  __m128 lo = _mm256_castps256_ps128(X);
  __m128 hi = _mm256_extractf128_ps(X, 1);
  __m128 sum4 = _mm_add_ps(lo, hi);

  sum4 = _mm_hadd_ps(sum4, sum4);
  sum4 = _mm_hadd_ps(sum4, sum4);

  return _mm_cvtss_f32(sum4);
}

void avx2_tmm(float* A, float* B, float* C,
  uint64_t N) {
    for (uint64_t i = 0; i < N; i++) {
      for (uint64_t j = 0; j < N; j++) {
        __m256 X = _mm256_setzero_ps();
        for (uint64_t k = 0; k < N; k += 8) {
          const __m256 AV = _mm256_loadu_ps(A + i * N + k);
          const __m256 BV = _mm256_loadu_ps(B + j * N + k);
          X = _mm256_fmadd_ps(AV, BV, X);
        }
        C[i * N + j] = hsum_avx(X);
      }
    }
}

int main() {
  int n = 1024;
  float* A = new float[n * n];
  float* B = new float[n * n];
  float* C = new float[n * n];

  init(A, n, n, false);
  init(B, n, n, false);
  init(C, n, n, true);

  auto start = chrono::high_resolution_clock::now();

  avx2_tmm(A, B, C, n);

  auto end = chrono::high_resolution_clock::now();
  chrono::duration<double> diff = end - start;
  cout << "Execution time: " << diff.count() << " s" << endl;
  cout << "C[0][0]: " << C[0] << endl;

  delete[] A;
  delete[] B;
  delete[] C;
}

Overwriting matmul_avx.cpp


In [43]:
!g++ -O2 -std=c++11 -mavx2 -mfma -march=native matmul_avx.cpp -o matmul_avx

In [44]:
!./matmul_avx

Execution time: 0.20872 s
C[0][0]: 264.403
